## Ejemplo práctico: conteo de palabras en tweets y elaboración de nube de palabras

En este documento se utiliza el fichero excel 'all_tweets.xlsx'. Se trata de una base de datos de tweets de una serie de medios de comunicación españoles que han usado hashtags relacionados con la crisis del coronavirus durante abril y mayo de 2020. Este fichero se ha generado mediante el acceso a la API de Twitter con un script en Python. A lo largo de este documento se explica el funcionamiento de un script para realizar un conteo de palabras de los tweets y elaborar una [una nube de palabras](https://es.wikipedia.org/wiki/Nube_de_palabras) con éstas.

Las tres siguientes celdas deben ser ejecutada en caso de que no estén instalados los paquetes `xlrd`, `pandas`, `matplotlib` o `wordcloud`. En caso de que ya estén instalados, simplemente se omiten.

In [ ]:
pip install xlrd

In [ ]:
pip install pandas

In [ ]:
pip install wordcloud

In [ ]:
pip install matplotlib

Lo primero es importar los paquetes que se pretende usar a lo largo del documento y definir la función auxiliar `get_col_index_by_name`. Esta función nos ayudará a identificar el índice de la columna que contiene los tweets.

In [ ]:
import xlrd
from collections import defaultdict
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def get_col_index_by_name(cols, name):
    
    i = 0
    for col in cols:
        if col.value == 'full_text':
            return i
        else:
            i += 1

    return None

En la siguiente celda se abre el fichero excel por la hoja que contiene los datos y se identifica el índice de la columna que contiene los tweets.

En el siguiente fragmento de código se usa el módulo `xlrd`:
* Sobre el [módulo xlrd](https://xlrd.readthedocs.io/en/latest/api.html#xlrd-sheet)

In [ ]:
file = 'all_tweets.xlsx'

wb = xlrd.open_workbook(file) 
sheet = wb.sheet_by_index(0)
index = get_col_index_by_name(sheet.row(0), 'full_text')

El siguiente bucle realiza las siguientes tareas:
1. Retira de cada tweet los saltos de línea, las palabras y ciertos caracteres que no nos interesan, como 'para', 'sobre' o los dos puntos (':').
1. Se divide cada tweet en palabras y se añaden a una lista.
1. Se añaden las palabras de cada tweet al objeto string `words_cloud` para formar la nube de palabras posteriormente.
1. Se hace un conteo de las palabras de cada tweet y se recoge la información en el diccionario `words`.


En este fragmento de código se utilizan métodos de la clase `str`, la función defaultdict() y comprehensiones de lista:
* Sobre [el método `str.replace()`](https://docs.python.org/3/library/stdtypes.html#str.replace).
* Sobre [el método `str.split()`](https://docs.python.org/3/library/stdtypes.html#str.split).
* Sobre [el método `str.lower()`](https://docs.python.org/3/library/stdtypes.html#str.lower).
* Sobre [la función `defaultdict()`](https://docs.python.org/3.8/library/collections.html#collections.defaultdict).
* Sobre las [comprehensiones de lista](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).

In [ ]:
tweets = sheet.col_values(index)[1:]
words = defaultdict(lambda:0)
words_cloud = ''
for tweet in tweets:
    
    tweet = tweet.replace('\n', ' ').lower()
    
    useless = [',', '.', ':', ';', 'para', 'según', 'por', 'desde', 'sobre', 'hasta', 'como', 'contra', 'este', 'tras']
    for uw in useless:
        tweet = tweet.replace(uw, '')
        
    tw_words = [word for word in tweet.split(' ') if len(word) > 3 and word[:4] != 'http' and word[0] != '@' and word[0] != '#']
    
    words_cloud += ' '.join(tw_words) + ' '
    
    for word in tw_words:
        words[word] += 1

Ya casi hemos terminado. El paquete `pandas` nos ofrece una gran variedad de herramientas para trabajar con datos, así que en el siguiente fragmento de código vamos a cambiar la estructura de nuestros datos para poder trabajar con objetos `DataFrame`. En el objeto `DataFrame` que hemos creado, la columna `word` contiene las palabras y la columna `n` contiene el número de veces que cada palabra aparece en los tweets.

En el siguiente fragmento se emplean comprehensiones de lista, la función `zip` y recursos propios del paquete `pandas`:
* Sobre [la función `zip`](https://docs.python.org/3/library/functions.html#zip).
* Sobre [el paquete `pandas`](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html).

In [ ]:
data_words = [x for x in zip(words.keys(), words.values())]

data = pd.DataFrame(data_words)
data.columns = ['word', 'n']

Veamos el resultado. El método `DataFrame.nlargest()` nos permite seleccionar las filas con los valores más altos de la variable que especifiquemos. En este caso le estamos indicando que nos muestre las 40 palabras que tienen un conteo más alto.

In [ ]:
print(data.nlargest(40, ['n']))

Una vez hemos visto las palabras más comunes procedemos a elaborar la nube de palabras. En el siguiente fragmento de código se usa `WordCloud` para definir las características de la nube y con `plt`, nuevo nombre con el que hemos definido `matplotlib.pylot` por comodidad, se construye el gráfico. Veamos cómo ha quedado.

En este fragmento se usan los siguientes paquetes:
* Sobre [el paquete `wordcloud`](https://pypi.org/project/wordcloud/)
* Sobre [el paquete `matplotlib`](https://matplotlib.org/)

In [ ]:
wordcloud = WordCloud(width = 800, 
                      height = 800, 
                      background_color ='white', 
                      min_font_size = 10).generate(words_cloud)

plt.figure(figsize = (7, 7), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0)
plt.show() 